In [1]:
# Step 1: Audio File Upload
from google.colab import files
import os

print("Upload the audio file here:")
uploaded = files.upload()

Upload the audio file here:


Saving call_112.mp3 to call_112.mp3


In [3]:
audio_file_name = next(iter(uploaded))
print(f"\nFile '{audio_file_name}' Audio file load successfully.")


File 'call_112.mp3' Audio file load successfully.


In [4]:
# Install Wishper Library
!pip install -U openai-whisper
import whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s

In [5]:
# Step 1.1: Convert Audio to english convertor  (Using Whisper)
print("\nWhisper Loading the model...")

model = whisper.load_model("base")

print("Audio Transcribe & Translate...")

# Output come in english only
result = model.transcribe(audio_file_name, task="translate")

# English convert text
english_text = result["text"]

print("\n--- English Transcription ---")
print(english_text)
print("--------------------------")


Whisper Loading the model...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 105MiB/s]


Audio Transcribe & Translate...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



--- English Transcription ---
 Santa's the emergency. What are you reporting? Hi, I need somebody escorted out of my store right away. He's causing problems. Okay, Mike is he fighting with people? Yeah, he's fighting with somebody right now. Is it a physical fight or verbal? Verbal. What is the address we need to come out to? Holy shit. What is the address? Or a gun. He has a gun or a knife. Okay. Stay on the phone. What's the address? Okay, okay. Okay. The address is 1330. Help with say-o, Dave Sarah, Cho'Ga. Now what makes you think he has a gun or a knife? Just hold it out. Okay. But there's a difference. If there's a gun. Okay. Stay on the phone. Go, go, go, go. In the bathroom. Go. Okay. And so this guy... Oh my god, he shot somebody! Oh my god! Oh my god! Oh my god! Oh my god. Holy shit. Okay. Okay. Sam, are you safe? Yeah, we're in the bathroom. Walk in the bathroom. Okay. So this guy is a white, black Hispanic or Asian. White. How old does he look? How old? What? How old does 

In [6]:
# Select the category (Using a Text Classification Model)

# Install Trnsformer library
!pip install transformers

In [7]:
from transformers import pipeline

print("\n Loading the text ...")

# Using zeroshot classificatio model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Defining categories
crime_categories = ['Robbery', 'Assault', 'Cybercrime', 'Theft', 'Domestic Violence', 'Fraud', 'Missing Person', 'Harassment']

print("Classifly model into categories ...")

# Time for Classification
result = classifier(english_text, crime_categories)


print("\n--- Complaint Category (Classifier Result) ---")
print(f"Most possible categories : {result['labels'][0]}")
print(f"Confidence Score: {result['scores'][0]:.2f}")
print("\nFull Result:")
print(result)
#print("------------------------------------------")


 Loading the text ...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Classifly model into categories ...

--- Complaint Category (Classifier Result) ---
Most possible categories : Domestic Violence
Confidence Score: 0.18

Full Result:
{'sequence': " Santa's the emergency. What are you reporting? Hi, I need somebody escorted out of my store right away. He's causing problems. Okay, Mike is he fighting with people? Yeah, he's fighting with somebody right now. Is it a physical fight or verbal? Verbal. What is the address we need to come out to? Holy shit. What is the address? Or a gun. He has a gun or a knife. Okay. Stay on the phone. What's the address? Okay, okay. Okay. The address is 1330. Help with say-o, Dave Sarah, Cho'Ga. Now what makes you think he has a gun or a knife? Just hold it out. Okay. But there's a difference. If there's a gun. Okay. Stay on the phone. Go, go, go, go. In the bathroom. Go. Okay. And so this guy... Oh my god, he shot somebody! Oh my god! Oh my god! Oh my god! Oh my god. Holy shit. Okay. Okay. Sam, are you safe? Yeah, we're in

In [8]:
!pip install -q spacy
!python -m spacy download en_core_web_sm

import spacy
import re
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
# Use Case To Detect Danger Level

high_threat_words = ["kill", "gun", "shoot", "bomb", "murder", "attack", "knife", "rape"]
medium_threat_words = ["beat", "threat", "follow", "scared", "stab", "danger"]
low_threat_words = ["scold", "argument", "push", "fight", "harass"]

def calculate_danger_score(text):
    text = text.lower()
    score = 1

    for word in high_threat_words:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            score += 2
    for word in medium_threat_words:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            score += 1
    for word in low_threat_words:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            score += 0.5

    return min(5, round(score))  # Max score 5


danger_score = calculate_danger_score(english_text)
print("Danger Level Score (1–5):", danger_score)

Danger Level Score (1–5): 5


In [16]:
#Use Case Summary Generation
!pip install -q transformers

from transformers import pipeline

# Load summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Generate summary
summary_result = summarizer(english_text, max_length=100, min_length=20, do_sample=False)

print("📝 USE CASE SUMMARY:")
print("--------------------------------------------------")
print(summary_result[0]['summary_text'])

Device set to use cpu


📝 USE CASE SUMMARY:
--------------------------------------------------
"Santa's the emergency" asks caller if he has a gun or a knife. "I want to get a really good description of this guy so we catch him" "He's wearing a giant sweater and a shark hat. A giant sweatshirt and an... What is it? Giant sweater and giant hat"


In [18]:
#Use Case Detect location

!pip install -q spacy
!python -m spacy download en_core_web_sm

import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")

# Optional: List of obviously invalid fake-location words
fake_locations = {'Santa', 'Today', 'Tomorrow', 'Someone', 'Anything', 'Nobody'}

def extract_locations_safe(text):
    doc = nlp(text)

    raw_locs = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC", "FAC"]]

    # Filter logic: remove short words, fake names, and duplicates
    clean_locs = []
    for loc in raw_locs:
        loc_strip = loc.strip()
        if len(loc_strip) > 3 and loc_strip not in fake_locations and loc_strip[0].isupper():
            clean_locs.append(loc_strip)

    return list(set(clean_locs)) if clean_locs else [np.nan]

# Run
locations = extract_locations_safe(english_text)
print("📍 Detected Location(s):", locations)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
📍 Detected Location(s): [nan]
